In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import utils as ut

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)

abastiment = pd.read_csv(Path(config['input_path']) / "abastiment_clean.csv", sep=",").fillna('')
pagesos    = pd.read_csv(Path(config['input_path']) / "pagesos_clean.csv", sep=",").fillna('')

In [3]:
abastiment.rename(columns={'PROJECTE':'MARCA'},inplace=True)
pagesos['dataset']    = 'pagesos'
abastiment['dataset'] = 'abastiment'

In [4]:
vdp = pd.read_csv(Path(config['input_path']
                           ) / 'vdp_clean.csv').fillna('')

In [5]:
# manually checked for names that where filtered as a match but weren't
not_duplicates = ['Celler Lo','Formatgeria Tiraval','Cor de fruita','Granja can Borrell','Maduixes de Munt']

matches = ut.get_project_matches(pagesos['MARCA'],abastiment['MARCA'],not_duplicates)

Partial match: BdeGust Beer Cervesa Compromesa vs. BdeGust Cervesa Compromesa 91 81
Partial match: BIOGRASSFEED vs. Biograssfed 96 91
Partial match: Tomating vs. Tomàting 88 88
Partial match: Ramaderia Ecològica ca l'Andreu vs. Ramaderia ecológica ca l’andreu 94 94
Partial match: L'Enllaç de l'Ebre vs. L'Enllaç de l’Ebre, SCCL 79 89
Partial match: (Mu)DA Vedella d'Agullana vs. MuDA. Vedella d'Agullana 94 96
Partial match: Granja de Montmelús vs. Granja i formatgeria de Montmelús 73 84
Partial match: Safra del Montsec vs. Safrà del Montsec 94 94
Partial match: Celler Optim vs. Celler Lo 76 89
Partial match: FORMATGERIA LAURA vs. Formatgeria Tiraval 78 82
Partial match: Cordú Fruits vs. Cor de fruita 80 83
Partial match: PLANESES AGRICULTIRA REGENERATIVA vs. Planeses Agricultura Regenerativa 97 97
Partial match: Granja Can Torrent vs. Granja can Borrell 83 83
Partial match: Maduixes del Maresme vs. Maduixes de Munt 72 81
Partial match: Can Terrades Produccio Agraria Ecològica sco vs. Ca

In [6]:
common_cols = [c for c in pagesos.columns if c in abastiment.columns]

In [7]:
all_data = pd.concat([pagesos[common_cols],
          abastiment[~(abastiment.MARCA.isin(matches.MARCA.values))][common_cols]],
          axis=0)

In [9]:
# find which names are in the dataset from the generalitat
# manually checked for names that where filtered as a match but weren't
not_duplicates = {'Nom productor':['CAN GEL SCP','CASA BERTRAN SCP','COLET VITICULTORS, SL',
                                   'COOPERATIVA AGRÍCOLA I CA SANT ISIDRE','ESPELT VITICULTORS',
                                   'GRANJA EL ROCALL','RAMADERIA AULINA SL','RAMADERIA CMC, SL',
                                   "FORMATGES ARTESANS DE PONENT SCP",
                                   'RAMADERIA ECOLOGICA, SL','RAMADERIA SOLÉ, SCP','SAT PASTURES DEL MONTSENY'],
                  'Marca Comercial':["AGROLI",'CAN GEL SCP', "CAL PUJOLET", "CAL ROIO", "CASA BERTRAN", 
                                     "CELLER COOPERATIU DE SALELLES", "FRUITES I VERDURES RAMS", 
                                     "GRANJA VIDAL", "HORT DE ANMART", "HORTA DE CAL VIU", "HORTALISSES PI", 
                                     "HORTS CA L'ALEGRIA", "OUS ECOLÒGICS DE L'EMPORDÀ", "GRANJA DEL MONTSENY", 
                                     "RAMADERIA AULINA", "RAMADERIA SOLÉ, SCP", "LA FORMATGERIA A TAULL", 
                                     "RAMADERIA CMC, SL","FORMATGES ARTESANS DE PONENT SCP",
                                     "COOPERATIVA AGRÍCOLA I CA SANT ISIDRE","GRANJA EL ROCALL, S.L.",
                                     "L'HORT DEL XIQUET", "HORTA DE BEA", "MAS LA MORA", 
                                     "COLET VITICULTORS, SL","RAMADERIA ECOLOGICA, SL",
                                     "ESPELT VITICULTORS, SL","GRANJA EL ROCALL, S.L.",
                                     "FRUITES I VERDURES E. CAMPOS", "LA COROMINA", "CAL MESTRET", 
                                     "EL REBOST DE L'ALEGRIA", "FRUITES I VERDURES PERE I ANA", 
                                     "L'HORT DEL MOLÍ", "MEL DE LA CONCA", "MAS EL PUIG", "L'HORT D'EN RIPOLL", 
                                     "RAIG D'ARBECA", "LA COROMINA", "GRANJA FELIP","SAT PASTURES DEL MONTSENY"]
                 }
vdp_matches = pd.DataFrame(columns=['exact','marca_inicial','MARCA'])

for col in ['Nom productor','Marca Comercial']:
    print('Searching matches in column:',col)
    matches = ut.get_project_matches(vdp['Nom productor'],all_data['MARCA'],not_duplicates[col])
    vdp_matches = pd.concat([vdp_matches, matches],axis=0)
    print("=================================\n\n")
    
vdp_matches = vdp_matches.drop_duplicates()
vdp_matches = vdp_matches.rename(columns={'marca_inicial':'marca_vdp'})
print('Number of coincidences with other datasets:',vdp_matches.shape[0])

Searching matches in column: Nom productor
Partial match: CAL CIGARRO, SL vs. Cal Cigarro SL 97 93
Partial match: CAL SERRATS, SCP vs. Cal Serrats SCP 97 93
Partial match: CAN GEL SCP vs. Can Gual SCP 87 82
Partial match: CAN GUAL, SCP vs. Can Gual SCP 96 92
Partial match: CAN MIMO, PASTORS I PRODUCTORS ARTESANS, S.L vs. Can Mimó, Pastors i productors artesans 92 97
Partial match: CAN MUNNAR CB vs. Can Munnar SC 92 92
Partial match: CAN TABAQUET, SCP vs. Cal Tabaquet 76 92
Partial match: CASA BERTRAN SCP vs. Casa Beta 72 89
Partial match: CELLER CEDO ANGUERA, S.L. vs. CELLER CEDÓ ANGUERA 82 95
Partial match: CELLER COOPERATIU GANDESA, SCCL vs. Celler Cooperatiu de Gandesa 85 89
Partial match: CENTRAL PARC DEL BAIX LLOBREGAT S.C.C.L. vs. Central Parc Baix Llobregat 81 85
Partial match: COLET VITICULTORS, SL vs. Mulet Viticultors 79 88
Partial match: COLLITA PRÒPIA SCP vs. Collita propia 81 93
Partial match: COOPERATIVA AGRÍCOLA I CA SANT ISIDRE vs. Cooperativa Agricola Sant Isidre Del P

In [ ]:
all_data = all_data.merge(vdp_matches[['marca_vdp','MARCA']],on='MARCA',how='left')

In [ ]:
all_data.to_csv(Path(config['input_path']) / 'all_data.csv', index=False)